In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import get_text
import validators
texts = get_text.get_texts("all")

We found a number of selections of texts, including books from Project Gutenberg and, and we created a dictionary of thoese texts in `get_text.py`, which contains helper functions to get those books as strings for analysis.

In [5]:
from get_text import books
keys = books.keys()
print(f"Texts in the dictionary are: {list(keys)}")

Texts in the dictionary are: ['Romeo and Juliet', "Finnegan's Wake", 'Great Gatsby', 'Canterbury Tales', 'Iliad', 'Monty Python']


If we want to add a limited selection of texts to our dictionary for analysis, or if we want to add additional texts to our we can use the function `add_text`.

In [6]:
limited_selections=["Monty Python", "Iliad", "Romeo and Juliet", "Great Gatsby"]
for selection in limited_selections:
    corpora = get_text.add_text(selection)

{'Romeo and Juliet': 'http://www.gutenberg.org/ebooks/1777.txt.utf-8', "Finnegan's Wake": 'https://archive.org/stream/finneganswake00joycuoft/finneganswake00joycuoft_djvu.txt', 'Great Gatsby': 'http://www.gutenberg.org/files/64317/64317-0.txt', 'Canterbury Tales': 'http://www.gutenberg.org/cache/epub/2383/pg2383.txt', 'Iliad': 'http://www.gutenberg.org/files/6150/6150-0.txt', 'Monty Python': 'https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/webtext/grail.txt'}


Next, we use functions from `helper.py` to give some data about our 

In [8]:
from urllib import request
import markov_with_syllables as markov
import syllablizer
analyze_text = "Romeo and Juliet"
MAX_WORDS = 10000

NameError: name 'markov' is not defined